In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
import dask.array as da
from dask import delayed
dask.config.set(scheduler='threads')

In [2]:
def extract_low_rating_restaurants(df, rating=3):
    res_df = df.loc[df["stars"] < rating, :]
    print("Le dataset filtré avec un rating inférieur à", rating, "présente", len(res_df), "lignes.")

In [3]:
def show_size_dataset(df):
    print("Le dataset présente", df.shape[0].compute(), "lignes et", df.shape[1], "colonnes.")

In [4]:
from dask.distributed import Client
client = Client(n_workers=8, threads_per_worker=1, processes=False, scheduler_port=0)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.1.17:8787/status,
Dashboard: http://192.168.1.17:8787/status,Workers: 8
Total threads: 8,Total memory: 46.82 GiB
Status: running,Using processes: False
Comm: inproc://192.168.1.17/39816/1,Workers: 8
Dashboard: http://192.168.1.17:8787/status,Total threads: 8
Started: Just now,Total memory: 46.82 GiB
Comm: inproc://192.168.1.17/39816/4,Total threads: 1
Dashboard: http://192.168.1.17:43951/status,Memory: 5.85 GiB
Nanny: None,


In [5]:
review = dd.read_json("yelp_dataset/yelp_academic_dataset_review.json",lines = True, encoding = 'utf-8', blocksize="100MB")
business = dd.read_json("yelp_dataset/yelp_academic_dataset_business.json",lines = True, encoding = 'utf-8', blocksize="100MB")

In [6]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [7]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [9]:
business = business.drop(columns=["address", "city", "state", "postal_code", "latitude", "longitude", "attributes", "hours", "is_open"])

In [11]:
restaurant_data = business.loc[(business["categories"].notnull()) & (business["categories"].str.contains("restaurant", case=False)), :]

In [12]:
restaurant_data = restaurant_data.drop(columns=["categories"])

In [13]:
for rate in [1, 1.5, 2, 2.5, 3]:
    extract_low_rating_restaurants(restaurant_data, rating=rate)

Le dataset filtré avec un rating inférieur à 1 présente 0 lignes.
Le dataset filtré avec un rating inférieur à 1.5 présente 247 lignes.
Le dataset filtré avec un rating inférieur à 2 présente 1728 lignes.
Le dataset filtré avec un rating inférieur à 2.5 présente 4639 lignes.
Le dataset filtré avec un rating inférieur à 3 présente 9489 lignes.


In [16]:
review = review.drop(columns=["user_id", "date", "review_id"])

In [22]:
restaurant_review = review.loc[review["business_id"].isin(restaurant_data["business_id"].compute()), :]

In [ ]:
for rate in [1, 1.5, 2, 2.5, 3]:
    extract_low_rating_restaurants(restaurant_review, rating=rate)

2023-05-16 16:56:45,041 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 4.87 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:56:45,212 - distributed.worker.memory - WARNING - Worker is at 88% memory usage. Pausing worker.  Process memory: 5.20 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:56:45,383 - distributed.worker.memory - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 5.21 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:56:45,544 - distributed.worker.memory - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 5.24 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:56:45,663 - distributed.worker.memory - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 5.24 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:56:45,820 - distributed.worker.memory - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 5.24 GiB -- Worker mem

Le dataset filtré avec un rating inférieur à 1 présente 0 lignes.


2023-05-16 16:58:26,046 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2023-05-16 16:58:26,066 - distributed.worker.memory - WARNING - Worker is at 96% memory usage. Pausing worker.  Process memory: 5.62 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:58:26,185 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2023-05-16 16:58:26,249 - distributed.worker.memory - WARNING - Worker is at 102% memory usage. Pausing worker.  Process memory: 6.00 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:58:26,357 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2023-05-16 16:58:26,362 - distributed.worker.memory - WARNING - Worker is at 102% memory usage. Pausing worker.  Process memory: 6.00 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 16:58:26,517 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU

Le dataset filtré avec un rating inférieur à 1.5 présente 567227 lignes.


2023-05-16 17:00:08,877 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2023-05-16 17:00:10,349 - distributed.worker.memory - WARNING - Worker is at 108% memory usage. Pausing worker.  Process memory: 6.35 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 17:00:11,004 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2023-05-16 17:00:11,114 - distributed.worker.memory - WARNING - Worker is at 153% memory usage. Pausing worker.  Process memory: 8.98 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 17:00:11,536 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-05-16 17:00:11,538 - distributed.worker.memory - WARNING - Worker is at 158% memory usage. Pausing worker.  Process memory: 9.30 GiB -- Worker memory limit: 5.85 GiB
2023-05-16 17:00:11,764 - distributed.utils_perf - WARNING - full garbage collections took 15% CP

In [ ]:
join = restaurant_data.merge(ddf_rev, how="inner", on="business_id")

In [ ]:
join.head()